In [32]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Display preferences
pd.set_option('Display.max_columns', 100)

In [22]:
# Create dataframe
diamonds = pd.read_csv('../data/diamonds.csv')

In [23]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 11 columns):
Unnamed: 0    53940 non-null int64
carat         53940 non-null float64
cut           53940 non-null object
color         53940 non-null object
clarity       53940 non-null object
depth         53940 non-null float64
table         53940 non-null float64
price         53940 non-null int64
x             53940 non-null float64
y             53940 non-null float64
z             53940 non-null float64
dtypes: float64(6), int64(2), object(3)
memory usage: 4.5+ MB


In [24]:
# Remove id
diamonds = diamonds.drop(['Unnamed: 0'], axis=1)

In [25]:
# Print unique values
categorical = ['cut', 'color', 'clarity']

for category in categorical:
    print(category.upper())
    print('{}'.format(diamonds[category].unique()))

CUT
['Ideal' 'Premium' 'Good' 'Very Good' 'Fair']
COLOR
['E' 'I' 'J' 'H' 'F' 'G' 'D']
CLARITY
['SI2' 'SI1' 'VS1' 'VS2' 'VVS2' 'VVS1' 'I1' 'IF']


In [26]:
# One-hot encoding
diamonds = pd.concat([diamonds, pd.get_dummies(diamonds['cut'], drop_first=True)], axis=1)
diamonds = pd.concat([diamonds, pd.get_dummies(diamonds['color'], drop_first=True)], axis=1)
diamonds = pd.concat([diamonds, pd.get_dummies(diamonds['clarity'], drop_first=True)], axis=1)

In [27]:
# Remove categorical variables
diamonds = diamonds.drop(categorical, axis=1)

In [31]:
diamonds.head()

,carat,depth,table,price,x,y,z,Good,Ideal,Premium,Very Good,E,F,G,H,I,J,IF,SI1,SI2,VS1,VS2,VVS1,VVS2
0,0.23,61.5,55.0,326,3.95,3.98,2.43,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
1,0.21,59.8,61.0,326,3.89,3.84,2.31,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0
2,0.23,56.9,65.0,327,4.05,4.07,2.31,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,0.29,62.4,58.0,334,4.20,4.23,2.63,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
4,0.31,63.3,58.0,335,4.34,4.35,2.75,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0


In [33]:
# Independent and dependent variables
X = diamonds.drop(['price'], axis=1)
y = diamonds['price']

In [34]:
# Standardize
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

## Clustering 

## Dimensionality Reduction 